In [ ]:
import numpy as np
import pandas as pd
from dowhy import CausalModel
import graphviz
import warnings
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor
# EconML imports
from econml.dml import LinearDML, CausalForestDML
from econml.cate_interpreter import SingleTreeCateInterpreter, SingleTreePolicyInterpreter
import seaborn as sns
import dowhy.plotter
%matplotlib inline
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')



# The Direct Acyclic Graph

G=graphviz.Digraph()
G.edge("ta" , "tr")
G.edge("vel" , "ta")
G.edge("rh" , "ta")
G.edge("ta" , "clo")
G.edge("ta" , "thermal_sensation")
G.edge("vel" , "tr")
G.edge("rh" , "tr")
G.edge("tr" , "clo")
G.edge("tr" , "thermal_sensation")
G.edge("vel" , "clo")
G.edge("vel" , "thermal_sensation")
G.edge("rh" , "clo")
G.edge("rh" , "thermal_sensation")
G.edge("met" , "thermal_sensation")
G.edge("clo" , "thermal_sensation")

######################################
#########################################

G.edge("door" , "vel")
G.edge("door" , "rh")
G.edge("door" , "thermal_sensation")




# print(dot.source)

G.format = 'pdf'
G.render(directory='DAG', view = False).replace('\\', '/')



In [2]:
# Reading the data
data = pd.read_excel("data.xlsx")

data_for_causal = data[["ta","tr", "vel", "rh", "met", "clo", "thermal_sensation", "door", "gender"]]
data_for_causal = data_for_causal.dropna()

In [3]:
data_for_causal.groupby('gender').size()

gender
female    551
male      557
dtype: int64

In [8]:
door = data_for_causal.copy()
door = door.loc[door['gender'] == 'female']
door = door.drop(labels='gender', axis=1)

In [9]:
data_for_causal.shape, door.shape

In [10]:
# Initializing causal model

model = CausalModel(data=door,
                     graph=G.source.replace("\t", ' ').replace("\n", ' '),
                     treatment="door",
                     outcome="thermal_sensation")

# Identifying the estimation method
identified_estimand= model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
   d                        
───────(E[thermalₛₑₙₛₐₜᵢₒₙ])
d[door]                     
Estimand assumption 1, Unconfoundedness: If U→{door} and U→thermal_sensation then P(thermal_sensation|door,,U) = P(thermal_sensation|door,)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!



In [11]:
random_state = 120
dml_estimate = model.estimate_effect(identified_estimand,
                                     method_name="backdoor.econml.dml.DML", # Calling EconMl double machine learning algorithm
                                     control_value = 1,
                                     treatment_value = 0,
                                     target_units = 'ate',
                                     confidence_intervals=False,
                                method_params={"init_params":{'model_y':GradientBoostingRegressor(random_state=random_state, learning_rate=0.0001),
                                                              'model_t': GradientBoostingRegressor(random_state=random_state, learning_rate=0.0001),
                                                              "model_final":LassoCV(fit_intercept=False, random_state=random_state),
                                                              'featurizer':PolynomialFeatures(degree=1, include_bias=True),
                                                              'random_state':random_state},
                                               "fit_params":{}})
print(dml_estimate.value)

-0.22326771296888534


In [ ]:
# Random cause
res_random=model.refute_estimate(identified_estimand, dml_estimate, method_name="random_common_cause", random_seed=123)
print(res_random)

# Add Unobserved Common Causes
res_unobserved=model.refute_estimate(identified_estimand, dml_estimate, method_name="add_unobserved_common_cause",
                                     confounders_effect_on_treatment="linear", confounders_effect_on_outcome="linear",
                                    effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02, random_seed=123)
print(res_unobserved)

# Placebo Treatment
res_placebo=model.refute_estimate(identified_estimand, dml_estimate,
        method_name="placebo_treatment_refuter", random_seed=123)
print(res_placebo)


# Data Subsets Validation
res_subset=model.refute_estimate(identified_estimand, dml_estimate,
        method_name="data_subset_refuter", subset_fraction=0.8,
        num_simulations=10, random_seed=123)
print(res_subset)